<a href="https://colab.research.google.com/github/rishav197/Multi-Object-Tracking/blob/main/HW4_MOT_SORT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **CV HW4: Multi-object Tracking (MOT) with Detection**
**Detection**: YOLOv5,
**Tracking**: Simple Online Realtime Tracking (SORT)

---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1. Unzip data folder**

In [ ]:
# Change the path according to your setup

!unzip '/content/drive/MyDrive/hw4/sort-master.zip'
!unzip '/content/drive/MyDrive/hw4/KITTI_17_images.zip'
!unzip '/content/drive/MyDrive/hw4/TrackEval-master.zip'

Archive:  /content/drive/MyDrive/hw4/sort-master.zip
 extracting: sort-master/.gitignore  
   creating: sort-master/data/
   creating: sort-master/data/train/
   creating: sort-master/data/train/ADL-Rundle-6/
   creating: sort-master/data/train/ADL-Rundle-6/det/
  inflating: sort-master/data/train/ADL-Rundle-6/det/det.txt  
   creating: sort-master/data/train/ADL-Rundle-8/
   creating: sort-master/data/train/ADL-Rundle-8/det/
  inflating: sort-master/data/train/ADL-Rundle-8/det/det.txt  
   creating: sort-master/data/train/ETH-Bahnhof/
   creating: sort-master/data/train/ETH-Bahnhof/det/
  inflating: sort-master/data/train/ETH-Bahnhof/det/det.txt  
   creating: sort-master/data/train/ETH-Pedcross2/
   creating: sort-master/data/train/ETH-Pedcross2/det/
  inflating: sort-master/data/train/ETH-Pedcross2/det/det.txt  
   creating: sort-master/data/train/ETH-Sunnyday/
   creating: sort-master/data/train/ETH-Sunnyday/det/
  inflating: sort-master/data/train/ETH-Sunnyday/det/det.txt  
   cre

# **2. Install requirements**

In [ ]:
!pip install pip==21.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [ ]:
!pip install scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install filterpy
!pip install ffmpeg
!pip install -r sort-master/requirements.txt
!pip install cv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 4.2 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110473 sha256=ba5d4338dc17c42e8f175bc5b0201be9739b59cafdeea59979653a3cabed1123
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=99950dfe53949159ab565547fc5e92040ab5b67f94ea509aa610463305a9b58d
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheel

In [ ]:
!pip install git+https://github.com/JonathonLuiten/trackeval.git
!pip install motmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JonathonLuiten/trackeval.git to /tmp/pip-req-build-1ig_jdnu
  Running command git clone --filter=blob:none -q https://github.com/JonathonLuiten/trackeval.git /tmp/pip-req-build-1ig_jdnu
  Resolved https://github.com/JonathonLuiten/trackeval.git to commit 12c8791b303e0a0b50f753af204249e622d0281a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trackeval: filename=trackeval-1.0.dev1-py3-none-any.whl size=149982 sha256=ad08fe18cb1c83f6dbf8e3ff9c54ecf04999e657b61d751d76a10eaf60daad44
  Stored in directory: /tmp/pip-ephem-wheel-cache-mxf2bo7u/wheels/1b/7e/e3/6002bff5f492c5182891101a29151436ce968d64ec97bff2fa
Successfully built trackeval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████████████████████

# **3. Import libraries**

In [ ]:
import torch
import torchvision
import cv2
import sys
sys.path.insert(0,'./sort-master/')
import matplotlib
from google.colab.patches import cv2_imshow
from collections import namedtuple, OrderedDict
import motmetrics as mm
import trackeval as te

# **4. Load YOLOv5 detector from torch hub**

In [ ]:
yolov5_detector = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)
yolov5_detector.float()
yolov5_detector.eval()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-5-8 Python-3.10.11 torch-2.0.0+cu118 CPU



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 14.1M/14.1M [00:00<00:00, 122MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

# **5. Import SORT library**

In [ ]:
sys.path.insert(0, "/content/sort-master")
from sort import *

#**6. Perform tracking with detection**

In [ ]:
from IPython.display import HTML
import PIL.Image
from base64 import b64encode
import seaborn as sns
import ffmpeg
import numpy as np
import os

In [ ]:
from PIL import Image
!pip install lap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached lap-0.4.0-cp310-cp310-linux_x86_64.whl


### loads and paths

In [ ]:
img_folder = "/content/KITTI_17_images"
# video_file = "/content/Outputvideo.mp4"
video_file = "/content/drive/MyDrive/hw4/outputVideoFile.mp4"


gt_file = "/content/drive/MyDrive/hw4/gt.txt"
out_file = "predict.txt"

In [ ]:
# create a video capture object
video_cap = cv2.VideoCapture(video_file)

img_files = sorted(os.listdir(img_folder))
# print(img_files)
images = [Image.open(os.path.join(img_folder, f)).convert('RGB') for f in img_files]
# print(images)



In [ ]:
fptr_out = open(out_file, "w")
fptr_out

<_io.TextIOWrapper name='predict.txt' mode='w' encoding='UTF-8'>

### Initializations tracker and tracked video

In [ ]:
obj_tracker = Sort()

# Create video write that used to combine tracked seq to form output video
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
final_out = cv2.VideoWriter("tracked_OutputVideo.mp4", fourcc, video_cap.get(cv2.CAP_PROP_FPS), (images[0].size[0], images[0].size[1]))


In [ ]:
# iterate over given sequence of frames of the video
for idx, img in enumerate(images):
  #Convert PIL image into OpenCV format
  image = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

  # Detections using yolov5 model
  res = yolov5_detector(image, size=640)
  detections = res.pandas().xyxy[0].values


  # obj detections for peoples/pedestrains only
  person_detections = []
  for detection in detections:
    if detection[5]==0: #check for person class (class_id = 0)
      person_detections.append(detection)

  #Here, we get tracked image by using a tracker(SORT)
  tracked_objs = obj_tracker.update(np.array(person_detections))

  #Combine the sequence of frames to produce a final tracked video with detection
  for person in tracked_objs:
    x, y, w, h, object_id = [int(x) for x in person]
    point1 = (x, y)
    point2 = (x+w, y+h)
    cv2.rectangle(image, point1, point2, (0,255,0), 2)
    cv2.putText(image, str(object_id), (x,y-10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 2)

    frame_count = idx+1
    conf_score, x, y, z = 1, 0, 0, 0
    fptr_out.write(f"{frame_count},{object_id},{x},{y},{x+w},{y+h},{conf_score},{x},{y},{z}\n")

  final_out.write(image)

# release the final tracked video
final_out.release()
fptr_out.close()

# **7. Report Evaluation Metrics**

In [ ]:
# Use the Track-Eval kit to report the complete set of performance and accuracy metrics
# Comment on and interpret MOTA and MOTP values

In [ ]:
!pip install numpy
!pip install scipy
!pip install pycocotools
!pip install matplotlib
!pip install opencv_python
!pip install scikit_image
!pip install pytest
!pip install Pillow
!pip install tqdm
!pip install tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from trackeval import utils
from trackeval import eval

In [ ]:
%cd /content/TrackEval-master/trackeval

/content/TrackEval-master/trackeval


In [ ]:
!pip install utils.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 154 kB 4.1 MB/s            
  Created wheel for utils.py: filename=utils_py-0.3.0-py3-none-any.whl size=12333 sha256=13fa1da3003c2efe1ff717a4feabd04d23ad45b515fd30a7eddbced73fef4cc2
  Stored in directory: /root/.cache/pip/wheels/66/77/84/11d6808ddbebd6f12cc980cc8ec8227d63cf8eb0743a115ac0
Successfully built utils.py


In [ ]:
%cd /content/TrackEval-master/trackeval

/content/TrackEval-master/trackeval


In [ ]:
!python eval.py motchallenge motmetrics --gt_path=/content/drive/MyDrive/hw4/gt.txt --res_path=/content/tracked_OutputVideo.mp4 -v

Traceback (most recent call last):
  File "/content/TrackEval-master/trackeval/eval.py", line 6, in <module>
    from . import utils
ImportError: attempted relative import with no known parent package


In [ ]:
import pandas as pd

In [ ]:
gt_content = np.loadtxt(gt_file, delimiter=',', skiprows=1)
# print(gt_content)

In [ ]:
gt = pd.DataFrame(gt_content, columns=['frame', 'id', 'x', 'y', 'w', 'h', 'class_id', 'visibility', 'unused', 'unused'])
gt

,frame,id,x,y,w,h,class_id,visibility,unused,unused
0,1.0,2.0,389.0,151.0,108.000,209.030,1.0,-1.0,-1.0,-1.0
1,1.0,6.0,485.0,145.0,16.667,55.039,0.0,-1.0,-1.0,-1.0
2,1.0,7.0,526.0,138.0,15.667,53.861,0.0,-1.0,-1.0,-1.0
3,1.0,8.0,545.0,143.0,14.667,47.346,0.0,-1.0,-1.0,-1.0
4,1.0,9.0,569.0,146.0,12.667,45.613,0.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
776,143.0,8.0,996.0,147.0,184.520,221.820,1.0,-1.0,-1.0,-1.0
777,144.0,7.0,850.0,102.0,212.060,266.880,1.0,-1.0,-1.0,-1.0
778,144.0,8.0,1017.0,148.0,193.920,220.660,1.0,-1.0,-1.0,-1.0
779,145.0,7.0,866.0,102.0,224.650,266.630,1.0,-1.0,-1.0,-1.0


In [ ]:
pred_content = np.loadtxt(gt_file, delimiter=',', skiprows=1)
# print(pred_content)

In [ ]:
pred = pd.DataFrame(pred_content, columns=['frame', 'id', 'x', 'y', 'w', 'h', 'class_id', 'visibility', 'unused', 'unused'])
pred

,frame,id,x,y,w,h,class_id,visibility,unused,unused
0,1.0,2.0,389.0,151.0,108.000,209.030,1.0,-1.0,-1.0,-1.0
1,1.0,6.0,485.0,145.0,16.667,55.039,0.0,-1.0,-1.0,-1.0
2,1.0,7.0,526.0,138.0,15.667,53.861,0.0,-1.0,-1.0,-1.0
3,1.0,8.0,545.0,143.0,14.667,47.346,0.0,-1.0,-1.0,-1.0
4,1.0,9.0,569.0,146.0,12.667,45.613,0.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
776,143.0,8.0,996.0,147.0,184.520,221.820,1.0,-1.0,-1.0,-1.0
777,144.0,7.0,850.0,102.0,212.060,266.880,1.0,-1.0,-1.0,-1.0
778,144.0,8.0,1017.0,148.0,193.920,220.660,1.0,-1.0,-1.0,-1.0
779,145.0,7.0,866.0,102.0,224.650,266.630,1.0,-1.0,-1.0,-1.0
